In [1]:
import numpy as np
import pandas as pd

from sklearn.svm import SVR

from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

from sklearn.metrics import mean_absolute_error

In [2]:
data = pd.read_csv('insurance.csv')
data.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [3]:
data.isnull().sum()

age         0
sex         0
bmi         0
children    0
smoker      0
region      0
charges     0
dtype: int64

In [4]:
encoder = OneHotEncoder(handle_unknown = 'ignore', sparse = False)

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


In [6]:
y = data.charges

X = data.drop('charges', axis = 1)

In [7]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size = 0.2)
X_train.head()

,age,sex,bmi,children,smoker,region
195,19,male,30.590,0,no,northwest
360,48,female,32.300,2,no,northeast
1150,18,female,30.305,0,no,northeast
294,25,male,26.800,3,no,southwest
512,51,male,22.420,0,no,northeast


In [8]:
features = X.columns.tolist()
features

['age', 'sex', 'bmi', 'children', 'smoker', 'region']

In [9]:
cats = [col for col in X.columns if X[col].dtype == 'object']
cats

['sex', 'smoker', 'region']

In [10]:
sub_t = X_train.select_dtypes(exclude = 'object')
sub_v = X_valid.select_dtypes(exclude = 'object')

sub_t.head()

,age,bmi,children
195,19,30.590,0
360,48,32.300,2
1150,18,30.305,0
294,25,26.800,3
512,51,22.420,0


In [11]:
OH_enc_train = pd.DataFrame(encoder.fit_transform(X_train[cats]))
OH_enc_valid = pd.DataFrame(encoder.transform(X_valid[cats]))

OH_enc_train.index = X_train.index
OH_enc_valid.index = X_valid.index


OH_enc_train

,0,1,2,3,4,5,6,7
195,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0
360,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
1150,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
294,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0
512,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
1079,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0
606,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
660,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0
894,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0


In [12]:
X_train = sub_t.join(OH_enc_train)
X_valid = sub_v.join(OH_enc_valid)

X_train.head()

,age,bmi,children,0,1,2,3,4,5,6,7
195,19,30.590,0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0
360,48,32.300,2,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
1150,18,30.305,0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
294,25,26.800,3,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0
512,51,22.420,0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0


In [13]:
y_train.head()

195      1639.56310
360     10043.24900
1150     2203.73595
294      3906.12700
512      9361.32680
Name: charges, dtype: float64

In [14]:
model = SVR()

model.fit(X_train, y_train)

SVR()

In [15]:
X_valid.head()

,age,bmi,children,0,1,2,3,4,5,6,7
1281,47,27.645,2,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
1325,61,33.535,0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0
1321,62,26.695,0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0
213,34,26.730,1,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
720,51,40.660,0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0


In [16]:
predicts = model.predict(X_valid)
predicts[:10]

array([9022.22756335, 9095.72266934, 9097.52732053, 8942.58576775,
       9047.27505265, 8884.27129037, 9083.02647986, 8960.78047026,
       8900.00583663, 9072.06217305])

In [17]:

mae = mean_absolute_error(y_valid, predicts)

print('MAE : ', mae)

MAE :  9443.948251919988
